In [ ]:
import os
print(os.getcwd())  # shows current working directory
print(os.listdir()) # shows files/folders in this directory

In [10]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
def load_and_split_documents(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [ ]:
extracted_data = load_and_split_documents('../data')
extracted_data

In [21]:
len(extracted_data)

637

In [22]:
from typing import List
from langchain.schema import Document

def filter_min_docs(docs: List[Document]) -> List[Document]:
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source", "")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [23]:
minimal_docs = filter_min_docs(extracted_data)